In [1]:
import cvxpy as cp
import scipy as sc
import numpy as np
import numpy.random as npr
import torch
from sklearn import datasets
import pandas as pd
import lropt
import matplotlib.pyplot as plt
import warnings
import seaborn
import imageio
warnings.filterwarnings("ignore")
plt.rcParams.update({
    "text.usetex":True,
    
    "font.size":18,
    "font.family": "serif"
})
colors = ["tab:blue", "tab:green", "tab:orange", 
          "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:grey", "tab:olive"]

In [234]:
# Formulate constants
n = 2
N = 50
# k = npr.uniform(1,4,n)
# p = k + npr.uniform(2,5,n)
npr.seed(1)
# P1, b1 = npr.rand(2,2), npr.rand(2)
# P2, b2 = npr.rand(2,2), npr.rand(2)
# P3, b3 = npr.rand(2,2), npr.rand(2)
# P3, b3 = npr.rand(2,2), npr.rand(2)
P1,P2 = np.eye(2), np.eye(2)
b1 = np.array([3.5,4])
b2 = np.array([4.,2.])

P1_tch, b1_tch = torch.tensor(P1, requires_grad=True), torch.tensor(b1, requires_grad=True)
P2_tch, b2_tch = torch.tensor(P2, requires_grad=True), torch.tensor(b2, requires_grad=True)


# Formulate loss function
def loss(t,x, data,l = 5): 
    return t + l*torch.mean(torch.maximum(
        torch.maximum(x@P1_tch@data.T + data @ b1_tch, x@P2_tch@data.T + data @ b2_tch) - t, 
        torch.tensor(0.,requires_grad = True))), t, torch.mean(torch.maximum(
        torch.maximum(x@P1_tch@data.T + data @ b1_tch, x@P2_tch@data.T + data @ b2_tch) - t, 
        torch.tensor(0.,requires_grad = True)))

def gen_demand(n, N):
    np.random.seed(7)
    F = np.random.normal(size = (n,2))
    sig = 0.1*F@(F.T)
    mu = np.random.uniform(-0.2,3,n)
    norms = np.random.multivariate_normal(mu,sig, N)
    d_train = np.exp(norms)
    return d_train

# Generate data
data = gen_demand(n,N)

### Train relevant sets

In [236]:
def build_prob(n = 2, data = data, A = None, b = None, loss = None):
    # Formulate uncertainty set
    u = lropt.UncertainParameter(n,
                            uncertainty_set=lropt.Ellipsoidal(p=2,
                                                        data=data, loss = loss, A= A, b = b))
    # Formulate the Robust Problem
    x = cp.Variable(n)
    t = cp.Variable()

    objective = cp.Minimize(t)

    constraints = [cp.maximum(x@P1@u + u @ b1, x@P2@u + u @ b2) <= t]
    constraints += [x >= 0]

    return lropt.RobustProblem(objective, constraints)


In [237]:
def g_cp(P1,P2,b1,b2,x,u):
    return cp.maximum(x@P1@u + u @ b1, x@P2@u + u @ b2)

In [238]:
def final_solve(A_final, b_final):
    n = 2
    u = lropt.UncertainParameter(n,
                            uncertainty_set=lropt.Ellipsoidal(p=2,
                                                        A = A_final, b = b_final))
    x = cp.Variable(n)
    t = cp.Variable()

    objective = cp.Minimize(t)

    constraints = [g_cp(P1,P2,b1,b2,x,u) <= t]
    constraints += [x >= 0]
    prob = lropt.RobustProblem(objective, constraints)
    prob.solve()
    # result3.reform_problem.solve()
    x_final = x.value
    t_final = t.value
    
    return x_final, t_final
                                

In [239]:
s = 13

prob = build_prob(loss = loss)
# # Train only epsilon
result = prob.train(eps = True, lr = 0.001, step=50, momentum = 0.8, optimizer = "SGD", initeps = 1, seed = s)
df_eps = result.df
# # Train A and b
init_A = sc.linalg.sqrtm(sc.linalg.inv(np.cov(data.T)))
init_b = -init_A@np.mean(data, axis=0)



# Train A and b
result1 = prob.train(lr = 0.001, step=50, momentum = 0.8, optimizer = "SGD", seed = s, initA = init_A, initb = init_b, save_iters = True)
df1 = result1.df
A_fin = result1.A
b_fin = result1.b
A1_iters, b1_iters = result1.uncset_iters


# Subsequent epsilon train
result2 = prob.train(eps = True, lr = 0.001, step=50, momentum = 0.8, optimizer = "SGD", seed = s, initA = A_fin, initb = b_fin, save_iters = True)
df_r1 = result2.df
A_fin_2 = result2.A
b_fin_2 = result2.b

In [240]:
x_opt_learned_2, t_opt_learned_2 = final_solve(A_fin_2, b_fin_2)
x_opt_learned, t_opt_learned = final_solve(A_fin, b_fin)
x_opt_base, t_opt_base = final_solve(init_A, init_b)

print("base x,t =", x_opt_base, t_opt_base)
print("optimal x,t =", x_opt_learned, t_opt_learned)
print("after eps train optimal x,t =", x_opt_learned_2, t_opt_learned_2)

base x,t = [-5.76444730e-10  6.00198091e-10] 29.291885382365695
optimal x,t = [-1.71470893e-09  1.63205116e-09] 24.30116300003996
after eps train optimal x,t = [-7.39756438e-10  1.18062888e-09] 25.163873814012643


In [257]:
# Populate Z Values (a 7x7 matrix) - For a circle x^2+y^2=z
offset = 2
x_min, x_max = np.min(data[:,0]) - offset, np.max(data[:,0]) + offset
y_min, y_max = np.min(data[:,1]) - offset, np.max(data[:,1]) + offset
n_points = 100
X = np.linspace(x_min,x_max,n_points)
Y = np.linspace(y_min,y_max,n_points)
x_mesh,y_mesh = np.meshgrid(X,Y)

def level_set(A_final, b_final, offset = 2, n_points = n_points, x_mesh = x_mesh, y_mesh = y_mesh):
    x_opt, t_opt = final_solve(A_final, b_final)
    unc_level_set = np.zeros((n_points,n_points))
    g_level_set = np.zeros((n_points,n_points))
    for i in range(n_points):
        for j in range(n_points):
            u_vec = [x_mesh[i,j], y_mesh[i,j]]
            unc_level_set[i,j] = np.linalg.norm(A_final @ u_vec  + b_final)
            g_level_set[i,j] = np.maximum(x_opt@P1@u_vec + u_vec @ b1, x_opt@P2@u_vec + u_vec @ b2) - t_opt

    return unc_level_set, g_level_set


unc_level_learned_2, g_level_learned_2 = level_set(A_fin_2, b_fin_2)
unc_level_learned, g_level_learned = level_set(A_fin, b_fin)
unc_level_base, g_level_base = level_set(init_A, init_b)



plt.figure(figsize=(15, 5))
plt.title("Original and reshaped sets")
# Set axis label for the contour plot
plt.xlabel(r"$u_1$")
plt.ylabel(r"$u_2$")

plt.contour(x_mesh,y_mesh,unc_level_base, [1], colors = ["tab:red"], label = "initial sets")
plt.contour(x_mesh,y_mesh,unc_level_learned, [1], colors = ["tab:orange"], label = "initial train A,b")
# plt.contour(x_mesh,y_mesh,unc_level_learned_2, [1], colors = ["tab:orange"], label = "final train")

plt.contourf(x_mesh, y_mesh,g_level_learned, alpha = 0.4)
# plt.contour(x_mesh,y_mesh,g_level_base, [0], colors = ["tab:red"])
# plt.contour(x_mesh,y_mesh,g_level_learned, [0, 1, 2], colors = ["tab:orange"])
bob = plt.contour(x_mesh,y_mesh,g_level_learned, alpha = 1)
plt.clabel(bob, inline=1, fontsize=30)
plt.scatter(data[:,0],data[:,1])
# plt.savefig("Reshaped_ctrs.pdf", bbox_inches='tight')
plt.colorbar()
plt.show()


In [ ]:
b1,b2

(array([0.4236548 , 0.64589411]), array([0.79172504, 0.52889492]))